In [2]:
import sys
print(sys.executable)

c:\Users\pc\AppData\Local\Programs\Python\Python312\python.exe


In [ ]:
import pandas as pd
import numpy as np
from copy import deepcopy

import sys
sys.path.append('./utils')
from utils import review_feature
rf = review_feature()

from pandas_profiling import ProfileReport

In [4]:
df = pd.read_csv('../output/Preprocessed_Reviews.csv').sort_values(by = ['product'], ignore_index = True)

In [5]:
df

,product,answer_option,label,review_len
0,Accucheck,Fast and accurate delivery,0,4
1,Accucheck,Expected a longer expire date. Your Product Li...,0,14
2,Accucheck,I liked the prompt service,0,5
3,Accucheck,Good product,0,2
4,Accucheck,I not needed,0,3
...,...,...,...,...
1650,shampoo,Its not much effective as it has been stated i...,0,12
1651,shampoo,Liked it very nicely working now my scalp is a...,1,11
1652,shampoo,its my regular choice,0,4
1653,shampoo,Good but not very effective,0,5


The goal is to derive meaningful insights from customer reviews by extracting and quantifying various linguistic and contextual features.
This section outlines the different attributes derived from review content. These features aim to encapsulate every critical perspective expressed in reviews, enabling a more precise and analytical evaluation. Quantifying these dimensions is essential for achieving higher accuracy in downstream tasks.
Noun Emphasis (Rn):
Nouns typically represent the key subjects in a sentence, making them highly valuable in determining the core message of a review. The presence and significance of nouns indicate the topical relevance of the feedback. To compute this, Part-of-Speech (POS) tagging was used to isolate nouns, and their weighted importance was calculated as:
<br>Score(Rn) = TFIDF(nouns) / TFIDF(all terms)

Sentiment Orientation (Rp):
This metric quantifies the emotional tone of the review on a scale from -1 (strongly negative) to +1 (strongly positive), identifying whether the review leans toward a positive or negative opinion.

Opinion Measure (Rs):
Subjectivity scores range from 0 to 1, indicating the degree to which a review expresses opinions (subjective) versus stating facts (objective).
Example:

"Bournvita tastes very good with milk" → Subjective

"Bournvita is brown in color" → Objective

Lexical Diversity (Rc):
This reflects the linguistic richness and complexity of a review by assessing the proportion of unique words it contains relative to the total unique vocabulary of the entire review set.
<br>Rc = Unique words in a review / Unique words in full corpus

Review Length (Rw):
Simply counts the number of words in a review. Longer reviews may provide more detail, while shorter ones could be concise yet insightful.

Semantic Categorization (Rd):
High-quality reviews often focus on product functionality, usage experience, and effectiveness. To detect such content, a domain-specific word bank was created to classify reviews into categories like product-focused, delivery-related, or customer service-based.
<br>We applied fuzzy string matching using Levenshtein distance to manage typographical errors. For instance, if a review says “My dilivery was on time,” it is still mapped to the keyword "delivery" due to spelling similarity.

VADER Sentiment Score (Rsc):
To enhance sentiment detection accuracy, we employed VADER (Valence Aware Dictionary for sEntiment Reasoning), which is tailored for analyzing informal text like social media posts. It detects emotional tone in slang, emojis (😂), emoticons (:), :D), and even distinguishes emphasis via capitalization (e.g., "I am SAD" vs "I am sad").
Scoring criteria:

Rsc ≥ 0.5 → Positive

-0.5 < Rsc < 0.5 → Neutral

Rsc ≤ -0.5 → Negative



In [ ]:
df['Rn'] = 0.0
df['Rp'] = 0.0
df['Rs'] = 0.0
df['Rc'] = 0.0
df['Rd'] = 0.0
df['Rsc'] = 0.0

In [9]:
df

,product,answer_option,label,review_len,Rn,Rp,Rs,Rc,Rd,Rsc
0,Accucheck,Fast and accurate delivery,0,4,0.0,0.0,0.0,0.0,0.0,0.0
1,Accucheck,Expected a longer expire date. Your Product Li...,0,14,0.0,0.0,0.0,0.0,0.0,0.0
2,Accucheck,I liked the prompt service,0,5,0.0,0.0,0.0,0.0,0.0,0.0
3,Accucheck,Good product,0,2,0.0,0.0,0.0,0.0,0.0,0.0
4,Accucheck,I not needed,0,3,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1650,shampoo,Its not much effective as it has been stated i...,0,12,0.0,0.0,0.0,0.0,0.0,0.0
1651,shampoo,Liked it very nicely working now my scalp is a...,1,11,0.0,0.0,0.0,0.0,0.0,0.0
1652,shampoo,its my regular choice,0,4,0.0,0.0,0.0,0.0,0.0,0.0
1653,shampoo,Good but not very effective,0,5,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
product_list = df['product'].unique()

In [11]:
for product in product_list:
    data = df[df['product']==product]
    unique_bag = set()
    for review in data['answer_option']:
        review = review.lower()
        words = review.split()
        unique_bag = unique_bag.union(set(words))

    for indx in data.index:
        review = data.at[indx, 'answer_option']
        df.at[indx, 'Rp'] = rf.polarity_sentiment(review)
        df.at[indx, 'Rs'] = rf.subjectivity_sentiment(review)
        df.at[indx, 'Rd'] = rf.service_tag(review)
        df.at[indx, 'Rsc'] = rf.slang_emoji_polarity_compoundscore(review)
        df.at[indx, 'Rc'] = float(len(set(review.split()))) / float(len(unique_bag))

    df.loc[df['product']==product, 'Rn'] = rf.noun_score(data['answer_option'].values).values

In [12]:
df

,product,answer_option,label,review_len,Rn,Rp,Rs,Rc,Rd,Rsc
0,Accucheck,Fast and accurate delivery,0,4,0.233709,0.300000,0.616667,0.005195,1.0,0.0000
1,Accucheck,Expected a longer expire date. Your Product Li...,0,14,0.673415,-0.100000,0.400000,0.016883,1.0,0.0000
2,Accucheck,I liked the prompt service,0,5,0.319747,0.600000,0.800000,0.006494,1.0,0.4215
3,Accucheck,Good product,0,2,0.546925,0.700000,0.600000,0.002597,0.0,0.4404
4,Accucheck,I not needed,0,3,0.000000,0.000000,0.000000,0.003896,1.0,0.0000
...,...,...,...,...,...,...,...,...,...,...
1650,shampoo,Its not much effective as it has been stated i...,0,12,0.500000,-0.300000,0.800000,0.028640,0.0,-0.3724
1651,shampoo,Liked it very nicely working now my scalp is a...,1,11,0.166375,0.690000,0.900000,0.026253,0.0,0.5709
1652,shampoo,its my regular choice,0,4,0.500000,0.000000,0.076923,0.009547,0.0,0.0000
1653,shampoo,Good but not very effective,0,5,0.000000,0.234615,0.607692,0.011933,0.0,-0.4032


In [15]:
df.to_csv('../output/Features.csv',index = False)